## Extract Zip files

In [3]:
import os
import zipfile

# Define the base directory where ZIP files are located
base_dir = "New SkinDisease"
extract_dir = os.path.join(base_dir, "extracted")

# Ensure the extracted directory exists
os.makedirs(extract_dir, exist_ok=True)

# Loop through all folders and extract ZIP files separately
for root, _, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".zip"):
            zip_path = os.path.join(root, file)
            folder_name = os.path.splitext(file)[0]  # Get ZIP filename without extension
            target_dir = os.path.join(extract_dir, folder_name)  # Create a separate folder
            os.makedirs(target_dir, exist_ok=True)  # Ensure folder exists

            with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                zip_ref.extractall(target_dir)  # Extract into separate folder
            print(f"Extracted: {file} → {target_dir}")

Extracted: AcneV2.v2-acnev2.yolov8.zip → New SkinDisease\extracted\AcneV2.v2-acnev2.yolov8
Extracted: Acne_Detection_Sai.v2i.yolov8.zip → New SkinDisease\extracted\Acne_Detection_Sai.v2i.yolov8
Extracted: eczema.v1i.yolov8 (1).zip → New SkinDisease\extracted\eczema.v1i.yolov8 (1)
Extracted: eczema.v1i.yolov8.zip → New SkinDisease\extracted\eczema.v1i.yolov8
Extracted: Detection of melasma.v1i.yolov8.zip → New SkinDisease\extracted\Detection of melasma.v1i.yolov8
Extracted: Melasma_detection.v1i.yolov8.zip → New SkinDisease\extracted\Melasma_detection.v1i.yolov8
Extracted: melasma_skripsi.v1i.yolov8.zip → New SkinDisease\extracted\melasma_skripsi.v1i.yolov8
Extracted: S-003.v1i.yolov8.zip → New SkinDisease\extracted\S-003.v1i.yolov8
Extracted: Rosacea.v3i.yolov8.zip → New SkinDisease\extracted\Rosacea.v3i.yolov8
Extracted: shing.v1i.yolov8.zip → New SkinDisease\extracted\shing.v1i.yolov8
Extracted: shingles.v2i.yolov8.zip → New SkinDisease\extracted\shingles.v2i.yolov8


## Step 1: Remove Augmented Images

✅ Why? To ensure that augmentation is applied consistently across all datasets.

Extract original (non-augmented) images from each ZIP file.

Keep only raw images for each skin condition.

### Scans

counting the number of images

displays hiearchy of the structure

In [29]:
import os

# Define the root directory to scan
root_dir = "New SkinDisease"

# ANSI escape codes for colors
CYAN = "\033[96m"
YELLOW = "\033[93m"
GREEN = "\033[92m"
BOLD = "\033[1m"
RESET = "\033[0m"

# Function to count files in a folder
def count_files(directory):
    return len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])

# Function to count total images and labels in a dataset folder
def count_dataset_items(dataset_path):
    total_images, total_labels = 0, 0
    for split in ["train", "valid", "test"]:
        split_path = os.path.join(dataset_path, split)
        images_path = os.path.join(split_path, "images")
        labels_path = os.path.join(split_path, "labels")

        if os.path.isdir(images_path):
            total_images += count_files(images_path)
        if os.path.isdir(labels_path):
            total_labels += count_files(labels_path)

    return total_images, total_labels

# Function to recursively scan and display folder hierarchy with image/label counts
def scan_folders(directory, indent=0):
    if not os.path.exists(directory):
        print(f"{YELLOW}⚠️ Directory not found!{RESET}")
        return

    for item in sorted(os.listdir(directory)):  # Sort for consistent order
        item_path = os.path.join(directory, item)
        prefix = " " * indent + "|-- "  # Indentation for hierarchy visualization
        
        if os.path.isdir(item_path):
            if item in ["images", "labels"]:  # Count files instead of listing them
                file_count = count_files(item_path)
                print(f"{CYAN}{prefix}📂 {BOLD}{item}{RESET} ({file_count} files)")
            else:
                # Count total images and labels for dataset folders
                total_images, total_labels = count_dataset_items(item_path)
                
                # Format the display string (hide 0 values)
                count_text = []
                if total_images > 0:
                    count_text.append(f"✅ {total_images} images")
                if total_labels > 0:
                    count_text.append(f"✅ {total_labels} labels")
                
                count_display = " ".join(count_text) if count_text else ""  
                print(f"{CYAN}{prefix}📂 {BOLD}{item}{RESET} {count_display}")
                
                scan_folders(item_path, indent + 4)  # Recursively scan subfolders
        else:
            print(f"{GREEN}{prefix}📄 {item}{RESET}")

# Run the folder scan
print(f"\n{BOLD}📁 Scanning folder hierarchy in:{RESET} {CYAN}{root_dir}{RESET}\n")
scan_folders(root_dir)



📁 Scanning folder hierarchy in: New SkinDisease

|-- 📂 Acne 
    |-- 📄 AcneV2.v2-acnev2.yolov8.zip
    |-- 📄 Acne_Detection_Sai.v2i.yolov8.zip
|-- 📂 Eczema 
    |-- 📄 eczema.v1i.yolov8 (1).zip
    |-- 📄 eczema.v1i.yolov8.zip
|-- 📂 Melasma 
    |-- 📄 Detection of melasma.v1i.yolov8.zip
    |-- 📄 Melasma_detection.v1i.yolov8.zip
    |-- 📄 S-003.v1i.yolov8.zip
    |-- 📄 melasma_skripsi.v1i.yolov8.zip
|-- 📂 Rosecea 
    |-- 📄 Rosacea.v3i.yolov8.zip
|-- 📂 Shingles 
    |-- 📄 shing.v1i.yolov8.zip
    |-- 📄 shingles.v2i.yolov8.zip
|-- 📂 cleaned 
    |-- 📂 Acne_0 ✅ 2053 images ✅ 2053 labels
        |-- 📂 test 
            |-- 📂 images (76 files)
            |-- 📂 labels (76 files)
        |-- 📂 train 
            |-- 📂 images (1908 files)
            |-- 📂 labels (1908 files)
        |-- 📂 valid 
            |-- 📂 images (69 files)
            |-- 📂 labels (69 files)
|-- 📂 extracted 
    |-- 📂 Acne_0 ✅ 460 images ✅ 460 labels
        |-- 📄 Acne_Detection_Sai.v2i.yolov8.zip
        |-- 📄 READM

In [23]:
import os

def read_roboflow_readme(root_folder):
    ignore_lines = [
        "Roboflow is an end-to-end computer vision platform that helps you",
        "* collaborate with your team on computer vision projects",
        "* collect & organize images",
        "* understand and search unstructured image data",
        "* annotate, and create datasets",
        "* export, train, and deploy computer vision models",
        "* use active learning to improve your dataset over time",
        "For state of the art Computer Vision training notebooks you can use with this dataset,",
        "visit https://github.com/roboflow/notebooks",
        "To find over 100k other datasets and pre-trained models, visit https://universe.roboflow.com"
    ]

    # ANSI escape codes for colors
    BLUE = "\033[94m"
    BOLD = "\033[1m"
    RESET = "\033[0m"

    for dataset_folder in os.listdir(root_folder):
        dataset_path = os.path.join(root_folder, dataset_folder)
        if os.path.isdir(dataset_path):  # Ensure it's a directory
            readme_path = os.path.join(dataset_path, "README.roboflow.txt")
            
            if os.path.exists(readme_path):  # Check if README file exists
                print(f"\n{BLUE}{BOLD}📂 Reading: {dataset_folder}/README.roboflow.txt 📄{RESET}\n")
                with open(readme_path, "r", encoding="utf-8") as f:
                    for line in f:
                        line = line.strip()
                        if not any(line.startswith(ignore) for ignore in ignore_lines):
                            print(line)  # Display filtered README contents  

                # Count images and labels
                total_images, total_labels = 0, 0
                for split in ["train", "valid", "test"]:
                    split_path = os.path.join(dataset_path, split)
                    images_path = os.path.join(split_path, "images")
                    labels_path = os.path.join(split_path, "labels")

                    if os.path.isdir(images_path):  # Check if images folder exists
                        total_images += len(os.listdir(images_path))
                    if os.path.isdir(labels_path):  # Check if labels folder exists
                        total_labels += len(os.listdir(labels_path))

                print(f"\n✅ {BOLD}Total Images:{RESET} {total_images}, {BOLD}Total Labels:{RESET} {total_labels}\n")

# Set the correct dataset directory path
root_folder = "New SkinDisease/extracted"  # Adjust this if needed
read_roboflow_readme(root_folder)



📂 Reading: Acne_0/README.roboflow.txt 📄


Acne_Detection_Sai - v2 2024-05-22 11:31am

This dataset was exported via roboflow.com on May 27, 2024 at 9:05 AM GMT




The dataset includes 2513 images.
Acne are annotated in YOLOv8 format.

The following pre-processing was applied to each image:
* Auto-orientation of pixel data (with EXIF-orientation stripping)
* Resize to 640x640 (Stretch)

The following augmentation was applied to create 3 versions of each source image:
* Equal probability of one of the following 90-degree rotations: none, clockwise, counter-clockwise
* Salt and pepper noise was applied to 0.12 percent of pixels



✅ Total Images: 2513, Total Labels: 2513


📂 Reading: Acne_1/README.roboflow.txt 📄


AcneV2 - v2 AcneV2

This dataset was exported via roboflow.com on February 20, 2025 at 4:23 PM GMT




The dataset includes 1282 images.
Pimple are annotated in YOLOv8 format.

The following pre-processing was applied to each image:
* Resize to 640x640 (Stretch)

The following

### Scanning augmented datas

In [22]:
import os
import cv2
import numpy as np

# Define dataset directories
dataset_path = r'New SkinDisease\extracted\Acne_0'
folders = ['train', 'valid', 'test']

def calculate_variation(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read image in grayscale
    if image is None:
        return 0  # If the image cannot be read, return 0 variation
    return np.var(image)  # Compute variance

total_variations = {}  # Dictionary to store total variations per folder
total_overall_variation = 0  # Initialize total variation across all folders
image_counts = {}  # Dictionary to count images per folder

# Iterate over each folder and compute total variation
for folder in folders:
    folder_path = os.path.join(dataset_path, folder, 'images')  # Path to images
    total_variation = 0  # Initialize total variation for this folder
    image_count = 0  # Counter for images processed

    if os.path.exists(folder_path):
        for filename in os.listdir(folder_path):
            if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):  # Ensure valid images
                continue

            image_path = os.path.join(folder_path, filename)
            variation = calculate_variation(image_path)
            total_variation += variation
            image_count += 1  # Increment image count

            # Debugging: Print variance for first 5 images
            if image_count <= 5:
                print(f'Image: {filename}, Variance: {variation}')

    total_variations[folder] = total_variation
    total_overall_variation += total_variation
    image_counts[folder] = image_count  # Store image count

# Print results
for folder, variation in total_variations.items():
    print(f'Total Variation in {folder}: {variation}, Images Processed: {image_counts.get(folder, 0)}')

print(f'Overall Total Variation: {total_overall_variation}')

# Prevent ZeroDivisionError
total_images = sum(image_counts.values())
if total_images > 0:
    avg_variation = total_overall_variation / total_images
    print(f'Average Variance per Image: {avg_variation:.2f}')
else:
    print("No images found. Check dataset path and file structure.")




Image: -17-8-66-14-53_jpg.rf.001f5545b92997e06b63bdde22b5dc03.jpg, Variance: 3417.8890908629837
Image: -17-8-66-14-53_jpg.rf.0eaa0aafc5c0c8ea8a202f1775d2d66b.jpg, Variance: 1285.0280627356528
Image: -17-8-66-14-53_jpg.rf.26f8790db7e042d9e30995df5d12d428.jpg, Variance: 1303.2653553361652
Image: -17-8-66-14-53_jpg.rf.2be70c254e361b4a079c4104e5efbae9.jpg, Variance: 3410.595512712002
Image: -17-8-66-14-53_jpg.rf.3fe40372bf9f7dd758b54af55e6d8d6f.jpg, Variance: 3013.2382797770447
Image: -17-8-66-14-53_jpg.rf.42d239f90464da74d3bf435d3e2044d6.jpg, Variance: 1259.7694958952907
Image: -17-8-66-14-53_jpg.rf.dcaa324cba018b79fcfce60540b874e0.jpg, Variance: 1259.7694958952907
Image: -_jpeg.rf.5b2afc489db70774581440d0866e2987.jpg, Variance: 677.7491543003978
Image: -_jpg.rf.2689f940f10f8f5fb3e5ac6dba0fa134.jpg, Variance: 2791.4742295607575
Image: -_jpg.rf.848428eecd6ff93777988cc1e4873a49.jpg, Variance: 3424.2230288829746
Image: -_jpg.rf.032bbb2ccc7ae8986e5391ac33c238b8.jpg, Variance: 3848.58285295628

### Scans Duplication

In [56]:
import os
from collections import defaultdict

# Define dataset path
dataset_path = r'New SkinDisease\extracted\Acne_0'
folders = ['train', 'valid', 'test']
image_exts = ('.jpg', '.png', '.jpeg')

# Dictionary to store duplicate counts
duplicate_counts = defaultdict(int)
duplicate_files = defaultdict(list)

# Process each folder
for folder in folders:
    image_folder = os.path.join(dataset_path, folder, 'images')

    if not os.path.exists(image_folder):
        continue

    filename_dict = defaultdict(list)  # Store all files with the same base name

    # Check all images in the folder
    for filename in os.listdir(image_folder):
        if not filename.lower().endswith(image_exts):
            continue  # Skip non-image files

        # Extract base filename (before .rf.<hash> part)
        base_name = filename.rsplit('_', 1)[0]  # Assumes filenames follow the pattern "name.rf.hash.jpg"

        filename_dict[base_name].append(filename)

    # Store duplicates
    for base_name, files in filename_dict.items():
        if len(files) > 1:
            duplicate_counts[folder] += len(files) - 1  # Count extra copies
            duplicate_files[folder].append((files[0], len(files)))  # Store example filename and count

# Display results
total_duplicates = 0
for folder, count in duplicate_counts.items():
    print(f'Folder: {folder} -> Duplicate Images Found: {count}')
    for file, dup_count in duplicate_files[folder]:
        print(f'  {file} = {dup_count} dup')
    total_duplicates += count

print(f'Total Duplicate Images Across All Folders: {total_duplicates}')


Folder: train -> Duplicate Images Found: 1908
  -17-8-66-14-53_jpg.rf.001f5545b92997e06b63bdde22b5dc03.jpg = 15 dup
  -66_jpg.rf.0c704fcdbd7fe5aba6c4b2121db80d9f.jpg = 3 dup
  -77_jpg.rf.04897c41af8190c8dfc1d9bb5e93c40d.jpg = 3 dup
  -8SFhXj7U7I_jpg.rf.6d783ef9e9598479c7afd277b959665f.jpg = 3 dup
  -QqvXU26S8A_jpg.rf.a2868f15676aebeb9f1cb5e3d78102be.jpg = 3 dup
  -_jpeg_jpg.rf.0d79e39c47dec03c2b93f2c7114aa7f6.jpg = 21 dup
  -_jpg.rf.008610d315027afe2f8068c2a45cc92d.jpg = 111 dup
  04yxunjT9g863MTSCqOU5A_jpg.rf.0101170d7d9c38e6378cd8b70f232932.jpg = 6 dup
  0e4f450a-fee3-4617-9f3a-b7347454b494_942113d8_jpg.rf.059b42a0f260b91caee3178752c394cb.jpg = 33 dup
  0_jpg.rf.26de499d7e73a5883ddd705c272f5a22.jpg = 3 dup
  101_jpg.rf.59caf14632dc258de7b2ea0d98bea097.jpg = 3 dup
  104_jpg.rf.5a520fde024dab8ba274df769613a102.jpg = 3 dup
  106_jpg.rf.21f85aabd9af1f636f44af853e7f535b.jpg = 3 dup
  1080x1350_0x0a330ca2_7403495401548913424_jpg.rf.052abdf98fd122cb10ef6b41ddf9c796.jpg = 3 dup
  108_jpg.rf.

### Removing Augmented Datas

In [27]:
import os
import shutil
from collections import defaultdict

# Define dataset paths
dataset_path = r'New SkinDisease/extracted/Acne_0'
cleaned_path = r'New SkinDisease/cleaned/Acne_0'
folders = ['train', 'valid', 'test']

# Ensure cleaned dataset structure exists
for folder in folders:
    os.makedirs(os.path.join(cleaned_path, folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(cleaned_path, folder, 'labels'), exist_ok=True)

# Function to extract base filename (excluding random hash)
def get_base_filename(filename):
    return filename.rsplit('_', 1)[0]  # Removes last underscore and hash

# Dictionary to store encountered images
image_registry = defaultdict(list)

# Iterate over dataset folders
duplicates_count = {folder: 0 for folder in folders}

for folder in folders:
    image_folder = os.path.join(dataset_path, folder, 'images')
    label_folder = os.path.join(dataset_path, folder, 'labels')
    
    if not os.path.exists(image_folder):
        continue  # Skip if folder doesn't exist
    
    for filename in os.listdir(image_folder):
        if not filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            continue  # Skip non-image files
        
        base_name = get_base_filename(filename)
        image_registry[base_name].append(filename)

# Process and move duplicates
for folder in folders:
    image_folder = os.path.join(dataset_path, folder, 'images')
    label_folder = os.path.join(dataset_path, folder, 'labels')
    
    cleaned_image_folder = os.path.join(cleaned_path, folder, 'images')
    cleaned_label_folder = os.path.join(cleaned_path, folder, 'labels')
    
    if not os.path.exists(image_folder):
        continue
    
    for base_name, filenames in image_registry.items():
        if len(filenames) > 1:
            duplicates_count[folder] += len(filenames) - 1  # Count duplicates removed
            
            # Retain one, move others to cleaned folder
            for file in filenames[1:]:
                src_image = os.path.join(image_folder, file)
                dst_image = os.path.join(cleaned_image_folder, file)
                
                if os.path.exists(src_image):
                    shutil.move(src_image, dst_image)
                    
                # Move corresponding label if exists
                label_file = os.path.splitext(file)[0] + '.txt'
                src_label = os.path.join(label_folder, label_file)
                dst_label = os.path.join(cleaned_label_folder, label_file)
                
                if os.path.exists(src_label):
                    shutil.move(src_label, dst_label)

# Print results
print("Duplicate Removal Summary:")
total_duplicates = 0
for folder, count in duplicates_count.items():
    total_duplicates += count
    print(f'Folder: {folder} -> Duplicate Images Removed: {count}')
print(f'Total Duplicates Removed: {total_duplicates}')

Duplicate Removal Summary:
Folder: train -> Duplicate Images Removed: 2053
Folder: valid -> Duplicate Images Removed: 2053
Folder: test -> Duplicate Images Removed: 2053
Total Duplicates Removed: 6159


### Scans for labels

Checks if label.txt have no value 0 kb or left blanked

In [38]:
import os
from termcolor import colored

def check_empty_labels(dataset_path):
    folders = ['train', 'valid', 'test']
    total_empty_labels = 0
    
    print(colored('\n🔍 Checking for empty label files...', 'cyan', attrs=['bold']))
    
    for folder in folders:
        label_folder = os.path.join(dataset_path, folder, 'labels')
        empty_files = []
        
        if os.path.exists(label_folder):
            for filename in sorted(os.listdir(label_folder)):  # Sorting filenames in ascending order
                file_path = os.path.join(label_folder, filename)
                if os.path.isfile(file_path) and os.path.getsize(file_path) == 0:
                    empty_files.append(filename)
        
        if empty_files:
            print(colored(f'📂 Folder: {folder} -> ❌ Empty Labels Found: {len(empty_files)}', 'red', attrs=['bold']))
            for file in empty_files:
                print(colored(f'   - {file}', 'yellow'))
            total_empty_labels += len(empty_files)
    
    print(colored(f'\n✅ Total Empty Labels Across All Folders: {total_empty_labels}', 'green', attrs=['bold']))

# Change this path accordingly
dataset_path = r'New SkinDisease/extracted/Acne_0'
check_empty_labels(dataset_path)



🔍 Checking for empty label files...

✅ Total Empty Labels Across All Folders: 0


### Move and Remove files

Move and remove files with no labels

In [37]:
import os
import shutil
from collections import defaultdict
from termcolor import colored

# Define dataset paths
dataset_path = r'New SkinDisease/extracted/Acne_0'
no_labels_path = r'New SkinDisease/No Labels/Acne_0'
folders = ['train', 'valid', 'test']

# Ensure No Labels dataset structure exists
for folder in folders:
    os.makedirs(os.path.join(no_labels_path, folder, 'images'), exist_ok=True)
    os.makedirs(os.path.join(no_labels_path, folder, 'labels'), exist_ok=True)

# Dictionary to track images with empty labels
empty_label_files = defaultdict(list)

# Scan for empty label files
for folder in folders:
    label_folder = os.path.join(dataset_path, folder, 'labels')
    image_folder = os.path.join(dataset_path, folder, 'images')

    if not os.path.exists(label_folder):
        continue  # Skip if folder doesn't exist

    for label_file in os.listdir(label_folder):
        label_path = os.path.join(label_folder, label_file)

        # Check if label file is empty
        if os.path.isfile(label_path) and os.path.getsize(label_path) == 0:
            base_name = os.path.splitext(label_file)[0]
            empty_label_files[folder].append(base_name)

# Move images and labels with empty labels
for folder in folders:
    image_folder = os.path.join(dataset_path, folder, 'images')
    label_folder = os.path.join(dataset_path, folder, 'labels')

    no_labels_image_folder = os.path.join(no_labels_path, folder, 'images')
    no_labels_label_folder = os.path.join(no_labels_path, folder, 'labels')

    if not os.path.exists(image_folder):
        continue

    for base_name in empty_label_files[folder]:
        image_extensions = ['.png', '.jpg', '.jpeg']
        for ext in image_extensions:
            src_image = os.path.join(image_folder, base_name + ext)
            dst_image = os.path.join(no_labels_image_folder, base_name + ext)

            if os.path.exists(src_image):
                shutil.move(src_image, dst_image)
                print(colored(f'🚀 Moved Image: {base_name + ext} -> {no_labels_image_folder}', 'yellow'))

        # Move empty label file
        src_label = os.path.join(label_folder, base_name + '.txt')
        dst_label = os.path.join(no_labels_label_folder, base_name + '.txt')

        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
            print(colored(f'📂 Moved Label: {base_name}.txt -> {no_labels_label_folder}', 'red'))

# Print Summary
total_moved = sum(len(files) for files in empty_label_files.values())
print(colored(f'\n✅ Total Images & Labels Moved to "No Labels": {total_moved}', 'green', attrs=['bold']))


🚀 Moved Image: -66_jpg.rf.0c704fcdbd7fe5aba6c4b2121db80d9f.jpg -> New SkinDisease/No Labels/Acne_0\train\images
📂 Moved Label: -66_jpg.rf.0c704fcdbd7fe5aba6c4b2121db80d9f.txt -> New SkinDisease/No Labels/Acne_0\train\labels
🚀 Moved Image: -_jpg.rf.008610d315027afe2f8068c2a45cc92d.jpg -> New SkinDisease/No Labels/Acne_0\train\images
📂 Moved Label: -_jpg.rf.008610d315027afe2f8068c2a45cc92d.txt -> New SkinDisease/No Labels/Acne_0\train\labels
🚀 Moved Image: 04yxunjT9g863MTSCqOU5A_jpg.rf.0101170d7d9c38e6378cd8b70f232932.jpg -> New SkinDisease/No Labels/Acne_0\train\images
📂 Moved Label: 04yxunjT9g863MTSCqOU5A_jpg.rf.0101170d7d9c38e6378cd8b70f232932.txt -> New SkinDisease/No Labels/Acne_0\train\labels
🚀 Moved Image: 0_jpg.rf.26de499d7e73a5883ddd705c272f5a22.jpg -> New SkinDisease/No Labels/Acne_0\train\images
📂 Moved Label: 0_jpg.rf.26de499d7e73a5883ddd705c272f5a22.txt -> New SkinDisease/No Labels/Acne_0\train\labels
🚀 Moved Image: 101_jpg.rf.59caf14632dc258de7b2ea0d98bea097.jpg -> New Skin

### Renaming the files